## Importing the necessary dependencies

In [2]:
import pandas as pd

C:\Users\nawaf\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

---

## Importing and preprocessing the data

In [10]:
kickoff_df = pd.read_csv('Datasets/kickoff.csv')
kickoff_df = kickoff_df.drop(columns=['Unnamed: 0'])

masters_df = pd.read_csv('Datasets/kickoff_with_masters.csv')
masters_df = masters_df.drop(columns=['Unnamed: 0'])

# stage1_df = pd.read_csv('Datasets/stage1.csv')
# stage1_df = stage1_df.drop(columns=['Unnamed: 0'])
stage1_df = pd.read_csv('Datasets/stage1_07_04_25.csv')
stage1_df = stage1_df.drop(columns=['Unnamed: 0'])

# total_df = pd.concat([masters_df, stage1_df], ignore_index=True)
total_df = pd.concat([kickoff_df, stage1_df], ignore_index=True)

In [11]:
total_df

,Team,Pistol Rounds Won,First Kills,KAST,Clutches,Eco,Semi-Eco,Half-Buy,Full-Buy,+1,...,+1 Opp,-1 Opp,0 Opp,+2 Opp,-2 Opp,-3 Opp,+3 Opp,+4 Opp,-4 Opp,Result
0,Evil Geniuses,2,23,0.67,0.068966,0.0,0.25,0.33,0.52,0.7200,...,0.8650,0.2800,0.6050,0.916667,0.046667,0.0,1.0,1.0,0.0,0
1,LOUD,2,26,0.71,0.173913,0.0,0.00,0.43,0.68,0.8650,...,0.7200,0.1350,0.3950,0.953333,0.083333,0.0,1.0,1.0,0.0,1
2,MIBR,1,11,0.59,0.040000,0.0,0.00,0.25,0.53,0.7000,...,0.7925,0.3000,0.6750,0.953333,0.166667,0.0,1.0,1.0,0.0,0
3,100 Thieves,3,28,0.82,0.200000,0.0,0.00,0.44,0.76,0.7925,...,0.7000,0.2075,0.3250,0.833333,0.046667,0.0,0.5,0.0,0.0,1
4,2Game Esports,2,20,0.67,0.071429,0.0,0.00,0.38,0.37,0.6550,...,0.9450,0.3450,0.9000,0.850000,0.110000,0.0,1.0,1.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,NRG Esports,1,12,0.67,0.037037,0.0,0.00,0.25,0.29,0.2750,...,1.0000,0.7250,0.8500,0.933333,0.000000,0.0,0.5,1.0,0.0,0
76,FURIA,0,13,0.62,0.206897,0.0,0.00,0.31,0.42,0.5250,...,0.7425,0.4750,0.4800,1.000000,0.333333,0.0,1.0,0.0,0.0,0
77,Leviatán,4,23,0.85,0.307692,0.0,1.00,0.90,0.63,0.7425,...,0.5250,0.2575,0.5200,0.666667,0.000000,0.0,1.0,1.0,0.0,1
78,Cloud9,3,36,0.74,0.131579,0.0,0.00,0.47,0.55,0.8400,...,0.8575,0.1600,0.6625,0.923333,0.090000,0.0,1.0,1.0,0.0,0


In [23]:
total_pistol_rounds = kickoff_df["Pistol Rounds Won"] + kickoff_df["Pistol Rounds Won Opp"]

kickoff_df["Pistol Rounds Won"] = kickoff_df["Pistol Rounds Won"] / total_pistol_rounds
kickoff_df["Pistol Rounds Won Opp"] = kickoff_df["Pistol Rounds Won Opp"] / total_pistol_rounds

In [12]:
total_pistol_rounds = total_df["Pistol Rounds Won"] + total_df["Pistol Rounds Won Opp"]

total_df["Pistol Rounds Won"] = total_df["Pistol Rounds Won"] / total_pistol_rounds
total_df["Pistol Rounds Won Opp"] = total_df["Pistol Rounds Won Opp"] / total_pistol_rounds

In [23]:
total_pistol_rounds = stage1_df["Pistol Rounds Won"] + stage1_df["Pistol Rounds Won Opp"]

stage1_df["Pistol Rounds Won"] = stage1_df["Pistol Rounds Won"] / total_pistol_rounds
stage1_df["Pistol Rounds Won Opp"] = stage1_df["Pistol Rounds Won Opp"] / total_pistol_rounds

In [14]:
df = total_df.copy()
total_df[["+4"]] = total_df[["+4"]].fillna(total_df[["+4"]].mean())
total_df[["-4"]] = total_df[["-4"]].fillna(total_df[["-4"]].mean())
# df[["+4", "-4"]] = df[["+4", "-4"]].fillna(0)

In [15]:
null_indices = total_df[total_df[["+4", "-4"]].isnull().any(axis=1)].index
total_df.loc[null_indices]

,Team,Pistol Rounds Won,First Kills,KAST,Clutches,Eco,Semi-Eco,Half-Buy,Full-Buy,+1,...,+1 Opp,-1 Opp,0 Opp,+2 Opp,-2 Opp,-3 Opp,+3 Opp,+4 Opp,-4 Opp,Result


---

## Training the Model

In [16]:
# Drop non-numeric or irrelevant columns (e.g., 'Team')
X = total_df.drop(columns=['Team', 'Result'])

# Target variable
y = total_df['Result']

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [36]:
# model = SVC(kernel='rbf', probability=True)

model = RandomForestClassifier(n_estimators=100, random_state=42)

# model = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=42)

model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [37]:
# Predict class labels and probabilities
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]  # probability of class 1 (win)

# Accuracy and classification report
print("Accuracy:", accuracy_score(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_proba))
print(classification_report(y_test, y_pred))

Accuracy: 0.8125
ROC AUC: 0.8571428571428572
              precision    recall  f1-score   support

           0       0.75      1.00      0.86         9
           1       1.00      0.57      0.73         7

    accuracy                           0.81        16
   macro avg       0.88      0.79      0.79        16
weighted avg       0.86      0.81      0.80        16



In [20]:
# Example: predict on the first 5 matches
probs = model.predict_proba(X.head(6))[:, 1]  # Probability of winning
print(probs)

[0.18 0.89 0.02 0.97 0.13 0.98]


In [15]:
model.predict_proba(X.head(5))

array([[0.87, 0.13],
       [0.08, 0.92],
       [1.  , 0.  ],
       [0.02, 0.98],
       [0.93, 0.07]])

---

## Extracting Team Names and Making Predictions

In [21]:
americas_teams_dict = {
    "100T": '100 Thieves',
    "2G": '2Game Esports',
    "C9": 'Cloud9',
    "EG": 'Evil Geniuses',
    "FUR": 'FURIA',
    "G2": 'G2 Esports',
    "KRU": 'KRÜ Esports',
    "LOUD": 'LOUD',
    "LEV": 'Leviatán',
    "MIBR": 'MIBR',
    "NRG": 'NRG Esports',
    "SEN": 'Sentinels',
}

In [32]:
# Specify the team and the columns you want to average
team_name_1 = americas_teams_dict["NRG"]
team_name_2 = americas_teams_dict["MIBR"]

columns_to_average = ['Pistol Rounds Won', 'First Kills', 'KAST', 'Clutches', 
                      'Eco', 'Semi-Eco', 'Half-Buy', 'Full-Buy',
                     '+1', '-1', '0', '+2', '-2', '-3', '+3', '+4', '-4']

# Column names for opponents' stats
columns_to_average_opp = [col + ' Opp' for col in columns_to_average]

# Extracting the specific teams from the DataFrame
# team_rows_1 = kickoff_df[kickoff_df['Team'] == team_name_1]
# team_rows_2 = kickoff_df[kickoff_df['Team'] == team_name_2]
team_rows_1 = total_df[total_df['Team'] == team_name_1]
team_rows_2 = total_df[total_df['Team'] == team_name_2]

# Calculating the mean for the selected columns
mean_values_1 = team_rows_1[columns_to_average].mean()
mean_values_2 = team_rows_2[columns_to_average].mean()

final_columns = ['Team'] + columns_to_average + columns_to_average_opp

# Combining team 1's stats with team 2's stats as opponent
row_1 = [team_name_1] + mean_values_1.tolist() + mean_values_2.tolist()

# Combining team 2's stats with team 1's stats as opponent
row_2 = [team_name_2] + mean_values_2.tolist() + mean_values_1.tolist()

result_df = pd.DataFrame([row_1, row_2], columns=final_columns)

result_df

,Team,Pistol Rounds Won,First Kills,KAST,Clutches,Eco,Semi-Eco,Half-Buy,Full-Buy,+1,...,Full-Buy Opp,+1 Opp,-1 Opp,0 Opp,+2 Opp,-2 Opp,-3 Opp,+3 Opp,+4 Opp,-4 Opp
0,NRG Esports,0.458333,26.833333,0.731667,0.111993,0.000000,0.141667,0.598333,0.473333,0.626667,...,0.576667,0.760000,0.225833,0.511852,0.928519,0.084074,0.005556,0.944444,0.736842,0.0
1,MIBR,0.509259,31.888889,0.726667,0.149723,0.055556,0.263333,0.467778,0.576667,0.760000,...,0.473333,0.626667,0.172500,0.430000,0.921111,0.070000,0.000000,1.000000,1.000000,0.0


In [43]:
X = result_df.drop(columns=['Team'])

probs = model.predict_proba(X)[:, 1]  # Probability of winning
print(probs)

[0.35 0.6 ]


Adjusting the predictions to fit probabilities...

In [75]:
round(probs[0] / (probs[0] + probs[1]), 3)

0.594

In [76]:
round(probs[1] / (probs[0] + probs[1]), 3)

0.406